# Chapter 12 Do not fall into the oblivion

To be able to restore and run multiple simulations the assets have to be saved somewhere. To choices arises. 
* The first one is to serialize the objects and save them as binary files.
* The second one is to save the information in a noSQL database.

Both options will be covered in the following lines.

## Serializing the objects

To directly save and load the objects we can use the module pickle.

First we will put all the code we need to create payoffs and options, as well as our factory class.

In [2]:
class PayOff:
    def __init__(self):
        pass
    
    def __call__(self,Spot):
        pass    

class PayOffBridge:        
    def __del__(self):
        del self
        
    def copy(self,InnerPayOff):
        from copy import deepcopy
        self = deepcopy(InnerPayOff)
        
class PayOffCall(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)
    
class VanillaOption:
    def __init__(self, ThePayOff_, Expiry_):
        self.__ThePayOff = ThePayOff_
        self.__Expiry = Expiry_
        
    def GetExpiry(self):
        return self.__Expiry
    
    def OptionPayOff(self, Spot):
        return self.__ThePayOff(Spot)

In [3]:
class PayOffFactory:
    def __init__(self):
        self.__TheCreatorFunctions = {}        
    
    def __del__(self):
        del self
    
    def RegisterPayOff(self, PayOffId, CreatorFunction):
        self.__TheCreatorFunctions[PayOffId] = CreatorFunction
        
    def CreatePayOff(self, PayOffId, *args, **kwargs):
        if PayOffId not in self.__TheCreatorFunctions.keys():
            print(f'{PayOffId} is an unknown payoff')
            return None
        else:
            return self.__TheCreatorFunctions[PayOffId](*args, **kwargs)   

We will create our option.

In [38]:
global thePayOffFactory  # We just the global keyword to be accesible from anywhere
thePayOffFactory  = PayOffFactory()
thePayOffFactory.RegisterPayOff("call",PayOffCall)

name = "call"
Maturity = 0.5
Strike = 40

thePayOFF = thePayOffFactory.CreatePayOff(name,Strike)
TheOption = VanillaOption(thePayOFF,Maturity)

We save our option.

In [5]:
import pickle
 
with open('TheOption', 'wb') as TheOption_file:
    pickle.dump(TheOption, TheOption_file)

We eliminate it.

In [6]:
del TheOption

We load it again.

In [8]:
with open('TheOption', 'rb') as TheOption_file:
    TheOption = pickle.load(TheOption_file)

In [18]:
print(thePayOFF)

With just 2 lines of code we can save and load a python object without any difficulty. We can create a saver class to input to other routines just like the MonteCarlo. Other possibilities include to create decorator functions for our classes to be able to save them without any problem.

## Using a noSQL DataBase

The problem gets tricky when we have to save tens of thousands of assets and we want to be able to move them quickly. Whereas the first alternative is easy to implement and do not require touching anything of the previous code, with this alternative we have to rewrite a good part of the code.

With the previous approach we do not need a factory, however with the following approach we will need a factory to construct the assets and the payoffs, since we are only going to store them as a JSON file into a MongoDB.

First we will rewritte the Payoffs class and the VanillaOption to get the atributes. Python has a default method for accessing the data members of one class via the __ dict __ attribute. However the output of that is not the optimal for our case so we will writte our own GetDict method that returns all the info as a dictionary.

In [90]:
class PayOffCall(PayOff,PayOffBridge):
    def __init__(self,Strike_):
        self.__Strike = Strike_
        
    def __call__(self,Spot):
        return max(Spot - self.__Strike,0)
    
    def GetDict(self):
        return {'Strike': self.__Strike}
    
class VanillaOption:
    def __init__(self, ThePayOff_, Expiry_):
        self.__ThePayOff = ThePayOff_
        self.__Expiry = Expiry_
        
    def GetExpiry(self):
        return self.__Expiry
    
    def OptionPayOff(self, Spot):
        return self.__ThePayOff(Spot)
    
    def GetDict(self):
        return {'Expiry': self.__Expiry, 'name':'PayOffCall', 'PayOffParameters':self.__ThePayOff.GetDict()}        

In [91]:
global thePayOffFactory  # We just the global keyword to be accesible from anywhere
thePayOffFactory  = PayOffFactory()
thePayOffFactory.RegisterPayOff("PayOffCall",PayOffCall)

thePayOFF = thePayOffFactory.CreatePayOff("PayOffCall",Strike)
TheOption = VanillaOption(thePayOFF,Maturity)
print(TheOption.GetDict())

{'Expiry': 0.5, 'name': 'PayOffCall', 'PayOffParameters': {'Strike': 40}}


Now we will save our data into a MongoDB.

In [110]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['MyDB']
Options = db.Options
data = TheOption.GetDict()
result = Options.insert_one(data)
print('One Option: {0}'.format(result.inserted_id))

One Option: 5ce43e01f704639cb20a848d


Our data has been saved without any problem and the ID of that option is showed next to One post: .
    
Now we will retrieve the information from the database.

In [123]:
MyOption = Options.find_one({'Expiry': 0.5})
print(MyOption)

{'_id': ObjectId('5ce43d09f704639cb20a847f'), 'Expiry': 0.5, 'name': 'PayOffCall', 'PayOffParameters': {'Strike': 40}}


Now we can use this technology to create a class that saves the information and another one that loads it.

In [152]:
class SaverDB:
    def __init__(self,DBName_,CollectionName_, host_ ='localhost', port_ = 27017):
        from pymongo import MongoClient
        self.__client = MongoClient(host_, port_)
        self.__db = self.__client[DBName_]
        self.__Collection = self.__db.CollectionName_
       
    def save(self, theOption_):
        result = self.__Collection.insert_one(theOption_.GetDict())
        print('One Option: {0}'.format(result.inserted_id))     

In [153]:
saver = SaverDB('MyDB',Options)
saver.save(TheOption)

One Option: 5ce44202f704639cb20a84a1


With this class we are able to save, doing one that loads it is just as straight forward.

With this last step we cover how to save options information.